In [1]:
import vaex
import pandas as pd 
import numpy as np
import os 
import dask
from datetime import datetime
import dask.dataframe as dd
from dask import delayed,compute
import sys 
current_dir = os.getcwd()
loading_dir = os.path.join(current_dir,"data","raw",
                           "flash_crash_DJIA","csv_files")
saving_dir = os.path.join(current_dir,"data","clean","flash_crash")
src_path = current_dir + "/src"
sys.path.append(src_path)

In [2]:
import loading_parallel

#### Testing at first the extraction function : 

In [3]:
ticker = "AAPL.OQ-2010"
bbo_files, trade_files = loading_parallel.extract_files(loading_dir,
                                       ticker,
                                       "2010-01-01",
                                       "2010-01-05")

In [11]:
print(trade_files)
print(bbo_files)

['/Users/ilyesbenayed/Desktop/Big data/data/raw/flash_crash_DJIA/csv_files/AAPL.OQ-2010/trade/2010_01/2010-01-01-AAPL.OQ-trade.csv.gz', '/Users/ilyesbenayed/Desktop/Big data/data/raw/flash_crash_DJIA/csv_files/AAPL.OQ-2010/trade/2010_01/2010-01-04-AAPL.OQ-trade.csv.gz', '/Users/ilyesbenayed/Desktop/Big data/data/raw/flash_crash_DJIA/csv_files/AAPL.OQ-2010/trade/2010_01/2010-01-05-AAPL.OQ-trade.csv.gz']
['/Users/ilyesbenayed/Desktop/Big data/data/raw/flash_crash_DJIA/csv_files/AAPL.OQ-2010/bbo/2010_01/2010-01-05-AAPL.OQ-bbo.csv.gz', '/Users/ilyesbenayed/Desktop/Big data/data/raw/flash_crash_DJIA/csv_files/AAPL.OQ-2010/bbo/2010_01/2010-01-04-AAPL.OQ-bbo.csv.gz', '/Users/ilyesbenayed/Desktop/Big data/data/raw/flash_crash_DJIA/csv_files/AAPL.OQ-2010/bbo/2010_01/2010-01-01-AAPL.OQ-bbo.csv.gz']


## The function works well : now time to test loadTRTH

#### This is an example on how to read one TRTH file : 

In [12]:
trth_file = loading_parallel.load_TRTH_trade(trade_files[1])
compute(trth_file)

(                                     trade_price  trade_volume
 xltime                                                        
 2010-01-04 09:30:00.576999424-05:00     213.5250           400
 2010-01-04 09:30:00.616000-05:00        213.5375          1130
 2010-01-04 09:30:00.635999488-05:00     213.5000           100
 2010-01-04 09:30:00.648999936-05:00     213.5500           100
 2010-01-04 09:30:00.676999680-05:00     213.5000           100
 ...                                          ...           ...
 2010-01-04 15:59:59.526999808-05:00     214.2900          1231
 2010-01-04 15:59:59.537999872-05:00     214.2800          2169
 2010-01-04 15:59:59.746000128-05:00     214.3300           700
 2010-01-04 15:59:59.828999680-05:00     214.2900           800
 2010-01-04 15:59:59.988999936-05:00     214.2700          1901
 
 [18379 rows x 2 columns],)

#### Example of reading too many files and concatenate them together : 

In [4]:
def read_TRTH_files(trade_files):
    delayed_dfs = []
    for file in trade_files:
        delayed_df = delayed(loading_parallel.load_TRTH_trade)(file,
                                              only_regular_trading_hours=True,
                                              merge_sub_trades=True,
                                              drop_columns=True)
        delayed_dfs.append(delayed_df)
    dfs = compute(*delayed_dfs)
    final_df = pd.concat(dfs, ignore_index=False)
    final_df = final_df.sort_index()
    return final_df

In [14]:
%%time 
read_TRTH_files(trade_files)

CPU times: user 8.24 ms, sys: 11.5 ms, total: 19.7 ms
Wall time: 459 ms


,trade_price,trade_volume
xltime,,
2010-01-04 09:30:00.576999424-05:00,213.5250,400
2010-01-04 09:30:00.616000-05:00,213.5375,1130
2010-01-04 09:30:00.635999488-05:00,213.5000,100
2010-01-04 09:30:00.648999936-05:00,213.5500,100
2010-01-04 09:30:00.676999680-05:00,213.5000,100
...,...,...
2010-01-05 15:59:59.070999296-05:00,214.3200,396
2010-01-05 15:59:59.480999424-05:00,214.3700,685
2010-01-05 15:59:59.482000384-05:00,214.3700,815


#### Remark: The function works well with multiple entries : 
Let's measure the time to read January month 

In [15]:
bbo_files, trade_files = loading_parallel.extract_files(loading_dir,
                                       ticker,
                                       "2010-01-01",
                                       "2010-01-31")

In [16]:
%%time 
read_TRTH_files(trade_files)

CPU times: user 41.7 ms, sys: 28.5 ms, total: 70.2 ms
Wall time: 1.01 s


,trade_price,trade_volume
xltime,,
2010-01-04 09:30:00.576999424-05:00,213.525000,400.0
2010-01-04 09:30:00.616000-05:00,213.537500,1130.0
2010-01-04 09:30:00.635999488-05:00,213.500000,100.0
2010-01-04 09:30:00.648999936-05:00,213.550000,100.0
2010-01-04 09:30:00.676999680-05:00,213.500000,100.0
...,...,...
2010-01-29 15:59:59.755999232-05:00,192.155000,300.0
2010-01-29 15:59:59.772000-05:00,192.000000,200.0
2010-01-29 15:59:59.813999616-05:00,192.185714,4048.0


#### Remark: Everithing works well, the program seems to be efficent : 
We read the data fast : (1 second to read more than 1  million data )

## BBO testing function 

In [17]:
bbo_example = loading_parallel.load_TRTH_bbo(bbo_files[0])
compute(bbo_example)

(                                     bid-price  bid-volume  ask-price  \
 xltime                                                                  
 2010-01-20 09:30:00.012998912-05:00     214.75           1     214.91   
 2010-01-20 09:30:00.015000576-05:00     214.80           1     214.93   
 2010-01-20 09:30:00.131000064-05:00     214.80           1     214.90   
 2010-01-20 09:30:00.143000320-05:00     214.80           1     214.93   
 2010-01-20 09:30:00.150999808-05:00     214.80           2     214.95   
 ...                                        ...         ...        ...   
 2010-01-20 15:59:59.785999872-05:00     211.64          26     211.65   
 2010-01-20 15:59:59.790999552-05:00     211.64          26     211.65   
 2010-01-20 15:59:59.889999360-05:00     211.64          26     211.65   
 2010-01-20 15:59:59.952999424-05:00     211.64          26     211.65   
 2010-01-20 15:59:59.996999424-05:00     211.64          26     211.65   
 
                                    

#### Now time to test on multiple bbo files 

In [5]:
def read_bbo_files(trade_files):
    delayed_dfs = []
    for file in trade_files:
        delayed_df = delayed(loading_parallel.load_TRTH_bbo)(file)
        delayed_dfs.append(delayed_df)
    dfs = compute(*delayed_dfs)
    final_df = pd.concat(dfs, ignore_index=False)
    final_df = final_df.sort_index()
    return final_df

In [19]:
%%time 
read_bbo_files(bbo_files)

CPU times: user 118 ms, sys: 112 ms, total: 229 ms
Wall time: 1.55 s


,bid-price,bid-volume,ask-price,ask-volume
xltime,,,,
2010-01-04 09:30:00.000999936-05:00,213.32,1,213.42,1
2010-01-04 09:30:00.009000192-05:00,213.37,1,213.50,3
2010-01-04 09:30:00.020000-05:00,213.38,2,213.50,1
2010-01-04 09:30:00.042000128-05:00,213.39,1,213.50,1
2010-01-04 09:30:00.087999744-05:00,213.40,1,213.50,1
...,...,...,...,...
2010-01-29 15:59:59.845999616-05:00,191.97,2,191.98,51
2010-01-29 15:59:59.889999360-05:00,191.97,2,191.98,50
2010-01-29 15:59:59.932999936-05:00,191.97,1,191.98,50


#### Everything seems good : We are well using the load_bbo files 

## Now the big function to save: 

In [6]:
@dask.delayed
def load_merge_trade_bbo(ticker,date_start,
                         date_end,
                         dirBase,
                         suffix="parquet",
                         suffix_save=None,
                         dirSaveBase = None ,
                         saveOnly=False,
                         doSave=False):
    """
    This function allows to merge trade and bbo together of a given ticker, 
    We should precise the destination to save the merged data 
    The date merged can be done in any period we would like to test on
    By default the method returns the merged series, but can save or only save the merged series 
    Gien a location of the saving directory, the suffix is by default a parquet file. 
    """
    bbo_files,trade_files  = loading_parallel.extract_files(dirBase,ticker,date_start,date_end)
    trades = read_TRTH_files(trade_files)
    bbos = read_bbo_files(bbo_files)
    try:
        trades.shape + bbos.shape
    except:
        return None
    
    events=trades.join(bbos,how="outer")
    
    if doSave:
        dirSave=os.path.join(dirSaveBase,ticker)
        if not os.path.isdir(dirSave):
            os.makedirs(dirSave)

        if suffix_save:
            suffix=suffix_save
        
        file_events=dirSave+"/"+date_end +"-"+ date_end +  "-events"+"."+suffix
       # pdb.set_trace()

        saved=False
        if suffix == "arrow":
            events=vaex.from_pandas(events,copy_index=True)
            events.export_arrow(file_events)
            saved=True
        
        elif suffix == "parquet":
            events.to_parquet(file_events,use_deprecated_int96_timestamps=True)
            saved = True
        
        if not saved:
            print(f"suffix {suffix}: format not recognized")

        if saveOnly:
            return saved
    return events




In [7]:

result = load_merge_trade_bbo(
    ticker="AAPL.OQ-2010",
    date_start="2010-01-01",
    date_end="2010-12-31",
    dirBase=loading_dir,
    suffix="arrow",
    suffix_save=None,
    dirSaveBase=saving_dir,
    saveOnly=False,
    doSave=True
)


In [8]:
%%time
compute(result)

CPU times: user 329 ms, sys: 1.11 s, total: 1.44 s
Wall time: 18.2 s


(#           trade_price    trade_volume    bid-price    bid-volume    ask-price    ask-volume    index
 0           nan            nan             213.32       1.0           213.42       1.0           2010-01-04 14:30:00.000999936
 1           nan            nan             213.37       1.0           213.5        3.0           2010-01-04 14:30:00.009000192
 2           nan            nan             213.38       2.0           213.5        1.0           2010-01-04 14:30:00.020000000
 3           nan            nan             213.39       1.0           213.5        1.0           2010-01-04 14:30:00.042000128
 4           nan            nan             213.4        1.0           213.5        1.0           2010-01-04 14:30:00.087999744
 ...         ...            ...             ...          ...           ...          ...           ...
 32,198,847  322.49         100.0           322.48       24.0          322.49       26.0          2010-12-31 20:59:59.733999872
 32,198,848  nan          

#### Now reading the file arrow: 

In [9]:
checking_dir = os.path.join(saving_dir,"AAPL.OQ-2010")

In [10]:
os.listdir(checking_dir)

['test1', '.ipynb_checkpoints', '2010-12-31-2010-12-31-events.arrow']

In [11]:
%%time

df=vaex.open(os.path.join(checking_dir,"2010-12-31-2010-12-31-events.arrow"))
df

CPU times: user 175 ms, sys: 112 ms, total: 287 ms
Wall time: 338 ms


#,trade_price,trade_volume,bid-price,bid-volume,ask-price,ask-volume,index
0,nan,nan,213.32,1.0,213.42,1.0,2010-01-04 14:30:00.000999936
1,nan,nan,213.37,1.0,213.5,3.0,2010-01-04 14:30:00.009000192
2,nan,nan,213.38,2.0,213.5,1.0,2010-01-04 14:30:00.020000
3,nan,nan,213.39,1.0,213.5,1.0,2010-01-04 14:30:00.042000128
4,nan,nan,213.4,1.0,213.5,1.0,2010-01-04 14:30:00.087999744
...,...,...,...,...,...,...,...
"32,198,847",322.49,100.0,322.48,24.0,322.49,26.0,2010-12-31 20:59:59.733999872
"32,198,848",nan,nan,322.48,24.0,322.49,16.0,2010-12-31 20:59:59.786999552
"32,198,849",322.49,1600.0,322.48,24.0,322.66,41.0,2010-12-31 20:59:59.798999552
"32,198,850",322.49,1093.0,nan,nan,nan,nan,2010-12-31 20:59:59.808000


### Little inspection of the data 

In [58]:
df.column_names

['trade_price',
 'trade_volume',
 'bid-price',
 'bid-volume',
 'ask-price',
 'ask-volume',
 'index']

In [64]:
import pytz
def convert_to_ny_timezone(dt):
    # Assuming the original datetime is in UTC, make it timezone aware
    utc_dt = pytz.utc.localize(dt)
    # Convert to 'America/New_York' timezone
    ny_dt = utc_dt.astimezone(pytz.timezone('America/New_York'))
    return ny_dt

In [73]:
#df['time_new'] = df['index'].apply(convert_to_ny_timezone)


In [53]:
df.column_names

['trade_price',
 'trade_volume',
 'bid-price',
 'bid-volume',
 'ask-price',
 'ask-volume',
 'index',
 '__index']

In [74]:
df.head()

#,trade_price,trade_volume,bid-price,bid-volume,ask-price,ask-volume,index,time_new
0,nan,nan,213.32,1,213.42,1,2010-01-04 14:30:00.000999936,2010-01-04 14:30:00.000999
1,nan,nan,213.37,1,213.5,3,2010-01-04 14:30:00.009000192,2010-01-04 14:30:00.009000
2,nan,nan,213.38,2,213.5,1,2010-01-04 14:30:00.020000,2010-01-04 14:30:00.020000
3,nan,nan,213.39,1,213.5,1,2010-01-04 14:30:00.042000128,2010-01-04 14:30:00.042000
4,nan,nan,213.4,1,213.5,1,2010-01-04 14:30:00.087999744,2010-01-04 14:30:00.087999
5,nan,nan,213.42,2,213.5,1,2010-01-04 14:30:00.095999744,2010-01-04 14:30:00.095999
6,nan,nan,213.42,1,213.5,1,2010-01-04 14:30:00.129999872,2010-01-04 14:30:00.129999
7,nan,nan,213.38,1,213.5,1,2010-01-04 14:30:00.140999936,2010-01-04 14:30:00.140999
8,nan,nan,213.41,2,213.5,1,2010-01-04 14:30:00.141999616,2010-01-04 14:30:00.141999
9,nan,nan,213.39,1,213.5,1,2010-01-04 14:30:00.148000,2010-01-04 14:30:00.148000


#### Last thing is to save: 

In [68]:
os.listdir(checking_dir)

['test1', '.ipynb_checkpoints', '2010-12-31-2010-12-31-events.arrow']

In [12]:
%%time
df.export(os.path.join(checking_dir,"test1",
                      '2010-12-31-2010-12-31-events.arrow'),
          compression="brotli")   # 20Gb uncompressed
         

CPU times: user 33.5 ms, sys: 603 ms, total: 636 ms
Wall time: 3.15 s


In [ ]:
df.dtypes